<a href="https://colab.research.google.com/github/pld000/z3/blob/main/z3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Теорем-прувер Z3 на Python

Z3 -- формально программа для автоматического доказательства теорем (theorem prover), однако она используется в повседневном программировании в самых разных целях, а особенно в проектах искусственного интеллекта. Z3 применяется и для решения самых разных логических и математических задач, и для автоматизации тестирования, и для верификации и синтеза программ и помощи в разработке сложных систем.

Z3 разработан и активно развивается научным подразделением Microsoft Research. Оригинальная версия собирается, конечно, в Visual Studio.
https://github.com/z3prover/z3

Мы будем использовать Z3Py -- обёртку на Python вокруг оригинального Z3. Для экспериментов с ней, и весьма сложных, достаточно colab-ноутбука.

Существуют также API к Z3 на Си/C++, .NET, Java и OCaml.

---

Материалы для самостоятельного изучения, ссылки на оригинальные ноутбуки:

https://ericpony.github.io/z3py-tutorial/guide-examples.htm

Programming Z3 https://theory.stanford.edu/~nikolaj/programmingz3.html

rise4fun https://rise4fun.com/z3/tutorialcontent/guide

Yurichev's book "SMT by Example"
https://yurichev.com/writings/SAT_SMT_by_example.pdf

---

*Подготовлено Высшей школой программирования Сергея Бобровского 2023*

https://vk.com/lambda_brain


In [ ]:
!pip install z3-solver # установка

In [ ]:
from z3 import * # подключаем все возможности

# 1. Первые шаги

## 1.1. Переменные

Например, имеется формула x + 2 * y = 7 , и мы хотим найти значения x и y, при которых она будет истинна.

Z3 это сделает автоматически, но мы должны ему объяснить исходные условия задачки.

Для этого первым шагом надо определить для солвера **переменные**, значениями которых будет манипулировать Z3.

Переменные для Z3 задаются типом и идентификатором.
Определим в формате Z3 две переменные x и y:



In [ ]:
x = Int('x')
y = Int('y')
type(x)

x и y -- это значения некоторого внутреннего типа `ArithRef` в Z3 (указатель на арифметический объект, выражение арифметического типа).

Проще всего считать, что x и y -- это целочисленные переменные (в системе типов Z3, конечно, а не Python). На самом деле x и y -- это некоторые целочисленные значения в математическом смысле, а не в вычислительном.

Тип `Int` -- это "тип" переменной, он используется для задания идентификаторов в коде Z3. Если требуется использовать значения (например, целочисленные константы), к названию типа добавляется `Val`. Константы Python приводятся к подходящим типам Z3 автоматически, но можно задавать их и явно.

In [ ]:
print(IntVal(123)) # целые
print(BoolVal(True)) # булевы
print(RealVal(2.5)) # вещественные
print(Q(1,3)) # иррациональные

Одной командой можно задавать сразу несколько переменных.

In [ ]:
a, b, c = Reals('a b c')
s, r = Ints('s r')

Конечно, стандартные функции Python тоже можно активно использовать.

In [ ]:
p = Real("p")
type(p) # какой тип этой переменной?
dir(p) # какие операции и функции над этим типом доступны?

## 1.2. Ограничения

Далее требуется задать так называемые **ограничения** (constraints), или условия, которых Z3 надо придерживаться при решении задачи. Чем больше ограничений, чем проще будет Z3 находить решение.

Например, мы хотим найти значения x и y для формулы x + 2 * y = 7 при условиях, что x > 2, а y < 10. Это выполнит функция `solve()`:


In [ ]:
solve(x > 2, y < 10, x + 2*y == 7) # решить данную систему из трёх ограничений

Ограничения задаются простым перечислением через запятую, как аргументы функции solve().

Важно понимать, что на самом деле `x + 2*y == 7` -- это тоже ограничение, точно такое же, как x > 2 и y < 10. Хотя для разработчика смысловая разница между ограничениями, конечно, имеется.

В ограничениях используются стандартные логические операции Python: `== != >= > <= <`

In [ ]:
# На самом деле, первые два ограничения не обязательны (но заранее мы этого не знали):
solve(x + 2 * y == 7)

In [ ]:
solve(x > 2) # будет найдено первое наименьшее подходящее значение

In [ ]:
# Изменим ограничения:
solve(x < 2, y < 10, x + 2*y == 7)

Z3 может конечно и не находить решения. В таких случаях говорят, что система ограничений нерешаемая, неистинная (unsatisfiable).

In [ ]:
x = Real('x')
solve(x > 4, x < 0) # no solution

In [ ]:
# Арифметические операции
print(x + x) # сложение
print(Sum([IntVal(i) for i in range(5)])) # Сумма значений списка
print(x * x) # умножение
print(x ** 4) # возведение в степень

# Логические операции
p = Bool('p')
q = Bool('q')
r = Bool('r')
print(And(p,q))
print(And(p,q,r)) # операция может быть применена к более чем двум аргументам
print(Or(p,q))
print(Implies(p,q))
print(Xor(p,q))

# Условия/ограничения
print(x == x) # равно
print(x != x) # не равно
print(x <= RealVal(3)) # неравенство

# ограничения -- это операции над булевыми выражениями
print(Or( x < 3,  x == 3, x > 3  ))

# И много всего другого, документация: s https://z3prover.github.io/api/html/namespacez3py.html

## 1.3. Упрощения

В дополнение к простому поиску значений Z3 также умеет выполнять **упрощения** выражений и ограничений (в функциональном программировании они называются **редукции**). Фактически работа с выражениями выполняется на символическом уровне.

Это естественная фича Z3, так как в процессе поиска значений гораздо удобнее и быстрее работать с упрощенными выражениями.

Упрощение выполняет функция `simplify()`.


In [ ]:
x = Int("x")
print(simplify(2*x + 2*x )) # 4*x
print(simplify(2*x - 2*x )) # 0

In [ ]:
# Более сложное выражение:
y = Int('y')
print(simplify(x + y + 2*x + 3))

In [ ]:
# Обратите внимание на результат:
print(simplify(x < y + x + 2)) # ожидается y > -2 ...

Z3 выводит результат в инвертированном виде. Not() -- это операция логического отрицания Z3, условно говоря, `not (y <= -2)` на Python.

In [ ]:
print(simplify(x + y + 2*x + 3))
print(simplify(x < y + x + 2))
print(simplify(And(x + 1 >= 3, x**2 + x**2 + y**2 + 2 >= 5)))

---

Команда `simplify` содержит набор готовых трансформаций для выражений Z3. Список её настроек можно получить вызовом `help_simplify()`

In [ ]:
help_simplify()

algebraic_number_evaluator (bool) simplify/evaluate expressions containing (algebraic) irrational numbers. (default: true)
arith_ineq_lhs (bool) rewrite inequalities so that right-hand-side is a constant. (default: false)
arith_lhs (bool) all monomials are moved to the left-hand-side, and the right-hand-side is just a constant. (default: false)
bit2bool (bool) try to convert bit-vector terms of size 1 into Boolean terms (default: true)
blast_distinct (bool) expand a distinct predicate into a quadratic number of disequalities (default: false)
blast_distinct_threshold (unsigned int) when blast_distinct is true, only distinct expressions with less than this number of arguments are blasted (default: 4294967295)
blast_eq_value (bool) blast (some) Bit-vector equalities into bits (default: false)
blast_select_store (bool) eagerly replace all (select (store ..) ..) term by an if-then-else term (default: false)
bv_extract_prop (bool) attempt to partially propagate extraction inwards (default: f

In [ ]:
x, y = Reals('x y')
t = simplify((x + y)**3, som=True) # что-то для работы с мономиалами
print(t)
t = simplify(t, mul_to_power=True) # использовать операцию возведения в степень
print(t)


In [ ]:
x, y = Reals('x y')
print(simplify(x == y + 2, arith_lhs=True))

## 1.4. Вычисления с рациональными и вещественными числами

В Python, да и во всех других массовых языках программирования имеется тип "вещественное число" (число с десятичной запятой), однако в Z3 принято представление таких чисел как **рациональных значений**, представленных дробью (числителем и знаменателем). Для этого используется функция `Q(числитель, знаменатель)`, а рациональный тип в Z3 называется `Real` для переменных (и соответственно `RealVal` для чисел). В качестве значения можно указывать обычное вещественное значение Python, однако оно будет преобразовано в рациональное.



In [ ]:
print(RealVal(3.14159))
print(RealVal(2.718281))

Z3 способен также в дополнение к рациональным числам работать с бесконечно длинными числами и иррациональными числами неограниченной точности (внутри Z3 они представляются, конечно, ограниченно). Иррациональные числа представляются при выводе как обычные вещественные числа с десятичной запятой.

In [ ]:
# Разрешение простого ограничения
x = Real("x")
solve(x * Q(17, 23) > 123.45)

Z3 поддерживает все основные математические операции, и умеет разбираться, конечно, не только с линейными выражениями, но и, например, с полиномиальными ограничениями.

In [ ]:
# Ограничения посложнее
x = Real('x')
y = Real('y')
solve(x**2 + 0.12 * y**2 > 3, x**3 + y < 5)

В случаях, когда значения переменных не удаётся выразить рациональным числом, используется вещественный формат. Так как значение иррациональное, в конце указывается ? (число продолжается дальше).

In [ ]:
x = Real('x')
y = Real('y')
solve(x**2 + y**2 == 3, x**3 == 2)

В коде можно комбинировать явные типы Z3 и числа Python, нужные преобразования типов выполнятся автоматически.

In [ ]:
print(1/3) # вещественное значение Python
print(RealVal(1)/3) # рациональное значение Z3

x = Real('x')

# все значения, используемые в выражениях с x, будут преобразованы к типам Z3
print(x + 1/3)
print(x + "1/3")
print(x + 0.25)


# 2. Арифметика

## 2.1. Приведения типов
Z3 может работать с веществеными/рациональными и целыми числами, которые могут перемешиваться, и Z3 автоматически выполняет нужные приведения типов.

In [ ]:
x = Real('x')
y = Int('y')
a, b, c = Reals('a b c')
s, r = Ints('s r')
print(x + y + 1 + (a + s))
print(ToReal(y) + c) # функция ToReal() выполняет приведение выражения к вещественному типу

In [ ]:
a, b, c = Ints('a b c')
d, e = Reals('d e')
solve(a > b + 2,
      a == 2*c + 10,
      c + b <= 1000,
      d >= e)


В Z3 имеются средства и для явного преобразования типов.

In [ ]:
x = Int("x")
y = Real("y")
ToReal(x)
ToInt(y)

## 2.2. Большие и "длинные" числа
Z3 может работать с бесконечно большими целыми числами и числами с бесконечным количеством знаков после запятой.

Z3 поддерживает алгебраические иррациональные числа, которые представляются как решения полиномиальных алгебраических уравнений, однако для наглядности они выводятся в обычной десятичной нотации.

Внутреннее представление некоторого значения можно получить методом sexpr(). Он отображает внутреннее представление Z3 для математических формул и выражений с помощью так называемых S-выражений (в стиле языка Lisp).


In [ ]:
x, y = Reals('x y')
solve(x + 10000000000000000000000 == y, y > 20000000000000000)

print(Sqrt(2) + Sqrt(3))
print(simplify(Sqrt(2) + Sqrt(3)))

# внутренние представления
print(simplify(Sqrt(2) + Sqrt(3)).sexpr())
print((x + Sqrt(y) * 2).sexpr())

## 2.3. Битовая арифметика

В Z3 имеется тип `битовые вектора` - BitVec, который позволяет явно задать количество бит, отводимых для хранения некоторого значения.

Например, функция BitVec('x', 16) создаёт переменную x (битовый вектор) длиной 16 бит. Целочисленные значения также можно создавать аналогично: BitVecVal(10, 4) создаёт битовый вектор размером 4 бита, хранящий значение 10.


In [ ]:
x = BitVec('x', 16)
y = BitVec('y', 16)
print(x + 2)
print((x + 2).sexpr()) # внутреннее представление битового вектора

print(simplify(x + y - 1))

a = BitVecVal(-1, 16) # битовые константы
b = BitVecVal(65535, 16)
print(simplify(a == b))

a = BitVecVal(-1, 32)
b = BitVecVal(65535, 32)
# для 32-битного числа со знаком значение -1 уже не "равно" значению 65535
print(simplify(a == b))

## 2.4. Битовые знаковые и беззнаковые операции

Z3 предлагает дополнительный набор операций для работы с беззнаковыми числами. Стандартные операции <, <=, >, >=, /, % и >> подразумевают работу с числами со знаками, а соответствующие им беззнаковые операции называются ULT, ULE, UGT, UGE, UDiv, URem и LShR.


In [ ]:
x, y = BitVecs('x y', 32)
solve(x + y == 2, x > 0, y > 0)


In [ ]:
# Битовые операции (в Z3 они называются операторы)
# & and
# | or
# ~ not
solve(x & y == ~y)

solve(x < 0)

# беззнаковая версия операции <
solve(ULT(x, 0))

Оператор >> -- это побитовый сдвиг вправо, и << это побитовый сдвиг влево.

Логический сдвиг вправо называется LShR.

In [ ]:
x, y = BitVecs('x y', 32)

solve(x >> 2 == 3)

solve(x << 2 == 3)

solve(x << 2 == 24)

## 2.5 Примеры

In [ ]:
x, y = Reals('x y')
solve(x >= 0, Or(x + y <= 2, x + 2*y >= 6), Or(x + y >= 2, x + 2*y > 4))

In [ ]:
x, y, s1, s2 = Reals('x y s1 s2')
solve(s1 == x + y, s2 == x + 2*y, x >= 0, Or(s1 <= 2, s2 >= 6), Or(s1 >= 2, s2 > 4))

# 3. Булева логика

## 3.1. Булев тип

Z3 поддерживает булев тип Bool (True/False), классические логические операции And, Or, Not, Implies (импликация, причина -> следствие), а также форму условного оператора If (if-then-else).

Логические операции записываются как функции, причём количество их аргументов может быть и больше двух. Например, And(p, q, True) означает на Python следующее: `p and q and True`


## 3.2. Решение булевых выражений

In [ ]:
q = Bool('q')
r = Bool('r')
solve(r == Not(q)) # найти значения r и q такие, что r == not q

In [ ]:
p = Bool('p')
q = Bool('q')
r = Bool('r')
solve(Implies(p, q), r == Not(q), Or(Not(p), r))

In [ ]:
p = Bool('p')
q = Bool('q')
r = Bool('r')
solve(Implies(p, q), r == Not(q), Or(Not(p), r))

Логические выражения можно конечно тоже упрощать.

In [ ]:
p = Bool('p')
q = Bool('q')
print(simplify(And(p, q, True))) # p and q and True == p and q
print(simplify(And(p, False))) # p and False == False

Более сложный пример, где комбинируются полиномиальные и логические ограничения.

In [ ]:
p = Bool('p')
x = Real('x')
solve(Or(x < 5, x > 10), Or(p, x**2 == 2), Not(p))

# 4. Дополнительное

## 4.1. Красивый вывод

Z3 предлагает режим вывода результата в формате HTML. Для этого используется команда `set_option()`, чей булев параметр html_mode определяет, выводится ли результат в нотации Z3Py (False) или в HTML (True).

In [ ]:
x = Int('x')
y = Int('y')
print(x**2 + y**2 >= 1)
set_option(html_mode=False) # вывод в стандартной нотации
print(x**2 + y**2 >= 1)

In [ ]:
set_option(html_mode=True) # вывод в нотации HTML
print(x**2 + y**2 >= 1)

Скопируем и вставим этот HTML-код:

x<sup>2</sup> + y<sup>2</sup> &ge; 1

## 4.2. Настройка среды Z3 с помощью set_option

`set_option()` позволяет также задавать и другие настройки среды выполнения Z3.

В частности, параметр `precision` определяет количество десятичных знаков после запятой для иррациональных значений.

In [ ]:
set_option(precision=5)
x = Real('x')
y = Real('y')
solve(x**2 + y**2 == 3, x**3 == 2)

Вывод рациональных чисел можно также выполнять в вещественном формате.

In [ ]:
# включить представление рациональных значений в формате с плавающей запятой
set_option(rational_to_decimal=True)
solve(3*x == 1)

# выключить ...
set_option(rational_to_decimal=False)
solve(6*x == 37)

------------------------------


## 4.3. Разбор и обход выражений

Каждое выражение в Z3 представляется в виде абстрактного синтаксического дерева AST, что естественно для всех подобных систем. Z3 предлагает набор функции для анализа и изучения таких деревьев.

In [ ]:
x = Int('x')
y = Int('y')

n = x + y >= 3 # сохраним выражение в переменной n

# количество аргументов (переменных) в выражении
print("num args: ", n.num_args()) # 2 - x и y

# в изучаемом выражении корень дерева -- операция >=
# у дерева два потомка: левая и правая части выражения

print("children: ", n.children()) # 2 потомка

print("1st child:", n.arg(0)) # левый потомок
print("2nd child:", n.arg(1)) # правый потомок
print("operator: ", n.decl()) # оператор-корень
print("op name:  ", n.decl().name()) # его наглядное имя

## 4.4. Строки

В дополнение к числовым и логическим значениям Z3 умеет также логически рассуждать и относительно строк (тип String). Для этого используется довольно богатый набор строковых операций, которые, как и операциии над другими типами, связываются логическими связками.

In [ ]:
s, t, u = Strings('s t u')
solve(And(PrefixOf(s, t), SuffixOf(u, t)), # s - префикс t; u - суффикс t
      And(Length(s) > 1, Length(u) > 1), # чтобы не генерировались пустые строки решения
      s != u)

In [ ]:
s, t = Consts('s t', SeqSort(IntSort()))
solve(Concat(s, Unit(IntVal(2))) == Concat(Unit(IntVal(1)), t))

## 4.5. Работа со списками

В Python традиционно хорошо развиты выразительные и лаконичные механизы работы со списками. Этот полезный подход активно задействован и в Z3. Если вы знакомились с функциональным программированием на моём курсе по F#, то наверняка помните, какое значение отводится спискам и различным рекурсивным приёмам их обработки. Теперь посмотрим на это направление с точки зрения математической логики.

In [ ]:
# создаём стандартный список питона со значениями 1,2,3,4,5
print([ x + 1 for x in range(5) ])

# создаём два списка из целочисленных переменных в формате Z3
X = [ Int('x%s' % i) for i in range(5) ]
Y = [ Int('y%s' % i) for i in range(5) ]
print(X)

# создаём список, равный сумме их значений
X_plus_Y = [ X[i] + Y[i] for i in range(5) ]

# в качестве результата будет, конечно, список из формул Z3,
# а не из конкретных "значений"
print(X_plus_Y)

# булева операция для связи двух списков
X_gt_Y = [ X[i] > Y[i] for i in range(5) ]
print(X_gt_Y)

# отрицание всех формул в списке целиком
print(And(X_gt_Y))

# матрица 3x3
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(3) ]
      for i in range(3) ]

pp(X) # специальная команда Z3 для форматированного вывода списков

Z3 также позволяет создавать одной операцией список (вектор) из заданного числа переменных целых, вещественных и булевых значений. В схеме автоматического формирования имён этих переменных в списках разберитесь самостоятельно.

In [ ]:
X = IntVector('x', 5) # список из пяти целых переменных
Y = RealVector('y', 3) # список из трёх вещественных переменных
P = BoolVector('p', 7) # список из семи булевых переменных
print(X)
print(Y)
print(P)
print([ y**2 for y in Y ])

print(Sum([ y**2 for y in Y ])) # на выходе получаем готовую формулу суммы всех элементов списка

В Z3 имеется тип `Seq` (последовательность), который задаёт потенциально бесконечную последовательность элементов любого типа. Например, тип String -- это последовательность 8-битных векторов. Для создания последовательности из одного элемента используется функция `Unit` с аргументом-значением.

In [ ]:
s, t = Consts('s t', SeqSort(IntSort())) # зададим две последовательности целых чисел
solve(Concat(s, Unit(IntVal(2))) == Concat(Unit(IntVal(1)), t)) # {s,2} == {1,t}

# 5. Солверы

Функция solve в Z3 может работать, используя различные решатели (солверы, solvers). По умолчанию используется собственный солвер Z3, но его можно при желании заменять на другой через Z3 API.


## 5.1. Универсальный солвер

Класс `Solver()` -- это универсальный многоцелевой солвер. Ограничения добавляются ему методом `add()` (говорят -- декларируются в решателе, asserted). Метод `check()` выполняет решение задекларированных ограничений. Результатом становится либо `sat` (истинно, satisfiable), если решение найдено, либо `unsat` (неистинно, unsatisfiable), если решения нету. Можно также сказать, что система декларируемых ограничений невыполнима. Кроме того, в процессе работы солвер может "запутаться" в рассуждениях и завершиться общей неудачей с неизвестным результатом `unknown`.

Для добавления и удаления декларируемых ограничений используются команды `push` и `pop`, а сам солвер хранит стек таких деклараций.


In [ ]:
x = Int('x')
y = Int('y')

s = Solver() # создаём объект-решатель

s.add(x > 10, y == x + 2) # добавляем ограничение
print(s) # будет выведено это ограничение

print("решение", s.check()) # ищем решение -- sat или unsat?

In [ ]:
s.push() # втолкнём текущее ограничение в стек солвера
s.add(y < 11) # добавим новое ограничение
print(s)
print(s.check()) # решения теперь нету -- unsat

s.pop() # удалим последнее новое ограничение
print(s)
print(s.check()) # sat

Пример, который Z3 не может решить.

In [ ]:
x = Real('x')
s = Solver()
s.add(2**x == 3)
print(s.check()) # unknown


## 5.2. Обход и анализ ограничений

Ограничения, задекларированные в солвере, можно перебирать, а также собирать статистики производительности для метода check.


In [ ]:
x = Real('x')
y = Real('y')
s = Solver()
s.add(x > 1, y > 1, Or(x + y > 3, x - y < 2))
for c in s.assertions(): # перебираем список из трёх ограничений
    print(c)

s.check()

print(s.statistics()) # общий список статистик по check

for k, v in s.statistics(): # выводим каждую статистику
    print("%s : %s" % (k, v)) # название : показатель производительности

## 5.3. Модель решения

Говорят, что Z3 ищет значения, удовлетворяющие множеству заданных ограничений.

**Решение** -- это модель для такого множества ограничений.
**Модель** -- это некоторая работающая интерпретация, которая делает каждое заданное ограничение истинным.


In [ ]:
x, y, z = Reals('x y z')

s = Solver()
s.add(x > 1, y > 1, x + y > 3, z - x < 10)
print(s.check()) # sat

m = s.model() # вытаскиваем из солвера модель правильного решения
print(m)

# чему равен x в решении?
print("x = %s" % m[x]) # интерпретация x в модели

for d in m.decls(): # перебираем все переменные модели
    print("%s = %s" % (d.name(), m[d]))


Ещё пример.

In [ ]:
x = Int("x")
y = Real("y")
z = Real("z")
p = Bool("p")
a = BitVec("a",32)

s = Solver()
s.add(x == 2)
s.add(y == 1/3)
s.add(p == True)
s.add(a == 0xDEADBEEF)
print(s.check()) # sat

m = s.model()

# методы приведения к конкретным типам:
print(m[x].as_long()) # длинное целое
print(m[y].numerator_as_long() / m[y].denominator_as_long()) # числитель / знаменатель
print(m[a].as_long())
print(bool(m[p])) # булево

m.eval(2*x + 1) # можно также вычислять выражения Z3 в контексте модели

s = Solver()
s.add(z ** 2 == 2)
print(s.check())
m = s.model()
print(m[z]) # иррациональный формат
print(m[z].approx()) # рациональный вид
print(m[z].approx().numerator_as_long() / m[z].approx().denominator_as_long())

Другой пример с булевой формулой.

In [ ]:
Tie, Shirt = Bools('Tie Shirt')
s = Solver()
s.add(Or(Tie, Shirt), Implies(Tie, Shirt), Implies(Shirt, Not(Tie)))
print(s.check())
print(s.model())

# 6. Функции

## 6.1. Немного о системе типов Z3

Сами по себе типы в Z3 называются sorts (сорта -- в смысле виды, классы). Для непосредственного задания сорта (например, типов параметров функций) используется обычная запись типа Z3, к которому добавлено Sort.

In [ ]:
# Объявления переменных Z3 разных типов (сортов)
x = Bool("x") # явный булев тип
x = Const("x" , BoolSort()) # то же самое, но по другому: переменная x типа BoolSort
p, q, r = Bools("p q r") # групповое определение нескольких переменных
x = Real("x")
y = Int("x")
v = BitVec("n", 32) # 32-битный вектор
f = FP("f", Float64()) # вещественный тип, с плавающей запятой
a = Array("a", IntSort(), BoolSort()) # массив (целый индекс - булево значение)
f = Function("f", IntSort(), BoolSort(), BoolSort()) # функция с двумя булевыми параметрами, возвращающая значение целого типа

In [ ]:
x = Const('x', IntSort())
print (eq(x, Int('x')))

a, b = Consts('a b', BoolSort())
print (And(a, b))

In [ ]:
# позапускайте эти примеры несколько раз, и сравните результаты
x = FP('x', FPSort(3, 4))
y = FP('y', FPSort(3, 4))
solve(10 * x == y)
solve(10 * x == y, x != 0)

Продвинутым пользователям также доступны алгебраические типы данных и ряд других сильных, но достаточно сложных фич.

## 6.2. Генерация имён переменных

В сложных задачах, когда количество переменных составляет сотни, тысячи и более, задавать имена вручную слишком трудоёмко.

Метод `FreshConst()` позволяет сгенерировать имя переменной автоматически (оно отличается обычно тем, что содержит в названии символ !). Первый параметр -- тип переменный, второй -- желаемый префикс.

Имеется сокращённая версия, когда после Fresh просто указывается нужный тип.

In [ ]:
print(FreshReal(prefix="x"))
print(FreshBool(prefix="b"))
print(FreshConst(IntSort() , prefix="f"))

## 6.3. Неинтерпретируемые фунции и константы

Z3 -- это по сути функциональный язык, где функции чистые (не имеют побочных эффектов) и тотальные (выдают результат за фиксированное время на всём диапазоне входных значений). В качестве математической основы в Z3 используется логика первого порядка (FOL).

В любом ограничении (например, x + y > 3), имеются идентификаторы, которые мы обычно называем переменные. Однако в FOL они называются **неинтерпретируемые константы**, что подразумевает, что допускается любая интерпретация этих идентификаторов, которая удовлетворяет ограничению x + y > 3.

И функции, и символьные константы в FOl считаются неинтерпретируемыми, что означает, что никакого смысла (интерпретации) исходно в них не вкладывается. Это прямое отличие от языков программирования, где например операция + имеет фиксированную стандартную интерпретацию (сложение). Неинтерпретируемость придаёт высокую гибкость процессу автоматического решения: допускается любая интерпретация, которая позволит соблюсти заданное ограничение.

Например, зададим две неинтерпретируемые целые константы x и y (переменные, по сути). И определим неинтерпретируемую функцию f, которая получает одним аргументом значение типа IntSort (Sort -- это не сортировка :) а "сорт" в смысле "тип"; IntSort -- это выражение целочисленного типа), и возвращает целое значение.


In [ ]:
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort())
solve(f(f(x)) == x, f(x) == y, x != y)


В качестве решения мы получаем не только значения переменных, но и решение (интерпретацию) для самой функции.

[x = 0, y = 1, f = [1 &rarr; 0, else &rarr; 1]]

Запись со стрелочками задаёт фунцию f() такую, что f(1) будет 0, а для всех остальных значений f() будет 1.

Тогда f(x=0) выдаст y=1, и f(y=1) выдаст x=0. Ограничения успешно выполнены.


In [ ]:
# то же с помощью класса солвера
x = Int('x')
y = Int('y')
f = Function('f', IntSort(), IntSort()) # задаём функцию f с целочисленными типами результата и параметра
s = Solver()
s.add(f(f(x)) == x, f(x) == y, x != y) # условие работы функции, и ограничения
print(s.check())

m = s.model()

# метод evaluate() выполняет вычисления с найденными результатами,
# в рамках текущего решения системы ограничений
print("f(f(x)) =", m.evaluate(f(f(x))))
print("f(x)    =", m.evaluate(f(x)))

Дополнительно про типы/сорты:

In [ ]:
Z = IntSort()
B = BoolSort()
f = Function('f', B, Z) # булев параметр, целый результат
g = Function('g', Z, B) # целый параметр, булев результат
a = Bool('a')
solve(g(1+f(a)))

In [ ]:
solve(And(g(1+f(a)), Not(g(2)), a))

In [ ]:
solve(And(g(1+f(a)), Not(g(2)), a, f(True) == 1))

## 6.4. Чуть-чуть о теории неинтерпретируемых функций

DeclareSort -- это свободный тип, не задающий каких-то конкретных требований к содержимому.


In [ ]:
S = DeclareSort('S') # S будет параметром функции условно произвольного типа

Определим унарную функцию f, получающую одно значение типа S, и возвращающую значение такого же типа S.

In [ ]:
f = Function('f', S, S)

Добавим переменную x типа S.

In [ ]:
x = Const('x', S)

Попробуем доказать следующее:

In [ ]:
solve(f(f(x)) == x, f(f(f(x))) == x) # sat

Таким образом мы получили множество эквивалентных **термов** `{ x, f(f(x)), f(f(f(x))) }`.

Но из f(f(x)) = x следует, что f(f(f(x)) = f(x) (если за x взять f(x)). Тогда мы получаем множество конгруэнтных термов `{ x, f(x), f(f(x)), f(f(f(x))) }`.

И далее мы можем расширять это множество до бесконечности: `{ x, f(x), f(f(x)), f(f(f(x))), f(f(f(f(x)))), f(f(f(f(f(x))))), ... }`. Это называется **конгруэнтное замыкание**.


In [ ]:
solve(f(f(x)) == x, f(f(f(x))) == f(x) ) # sat

Как это можно строго доказать? Так как мы расширяем множество одной подстановкой f(x)=x, попробуем найти контрпример (этой и другим тактикам решения и доказательства посвящён отдельный ноутбук). Для этого попытаемся найти такое значение функции f, при котором f(x)=x не выполнится.

In [ ]:
 solve(f(f(x)) == x, f(f(f(x))) == x, f(x) != x)

Z3 ответит, что `no solution`, из чего и следует, что наша гипотеза об эквивалентном множестве истинна.

## 6.5. Истинность и валидность

Формула/ограничение F считается валидной, если она истинна для **всех** значений её неинтерпретируемых символов.

Формула/ограничение F считается истинной/решённой, если она истинна для **некоторых** значений её неинтерпретируемых символов.

Валидность -- про поиск доказательства истинности формулы в целом.

Истинность -- про поиск конкретного решения для системы ограничений.

Напрямую с помощью солвера мы не можем определить валидность формулы в Z3 -- только её истинность. Простой способ проверять валидность -- это пытаться найти решение для отрицания формулы. Если оно найдено, сначит формула не валидна, и наоборот. Как это делать, рассказывается в следующих занятиях.

In [ ]:
p, q = Bools('p q')
demorgan = And(p, q) == Not(Or(Not(p), Not(q))) # p and q == not ((not p) or (not q))
print(demorgan)

def prove(f): # доказательство валидности формулы-параметра
    s = Solver()
    s.add(Not(f))
    if s.check() == unsat: # если отрицание формулы не доказывается,
        print("proved") # значит формула валидна
    else:
        print("failed to prove") # иначе невалидна

prove(demorgan)

# 7. Простые физические задачи

## 7.1. Школьная задачка по физике

В школе мы проходили уравнения, связывающие пройденное расстояние d, время t, ускорение a и начальную и конечную скорости движения v1 и v2. Например, мотоциклист едет со скоростью 30 м/с, на светофоре загорается жёлтый, он начинает тормозить с ускорением -8 м/с^2. Надо найти расстояние, которое он пройдёт до полной остановки.

In [ ]:
d, a, t, v1, v2 = Reals('d a t v1 v2') # объявляем наши переменные

# система ограничений
equations = [
   d == v1 * t + (a*t**2)/2, # зависимость расстояния от ...
   v2 == v1 + a*t, # связь между начальной и конечной скоростями
]
print("Условия:")
print(equations)

# исходные данные
problem = [
    v1 == 30,
    v2 == 0,
    a   == -8
]
print("Задача:")
print(problem)

print("Решение:")
solve(equations + problem)

## 7.2. Вторая задачка по физике

Мотоциклист стоит на светофоре. Как только загорается зелёный, он стартует с места с ускорением 6 м/с^2 на протяжении 4,1 секунды. Определите расстояние, которое мотоцикл преодолеет за это время.


In [ ]:
# переменные и equations сохраним из предыдущего блока.

# новые вводные
problem = [
    v1 == 0,
    t   == 4.10,
    a   == 6
]

# выведем результат в общепринятом формате с десятичным разделителем
set_option(rational_to_decimal=True)

solve(equations + problem)

# 8. Головоломки

## 8.1. Кот, пёс и мышь

Классическая головоломка. У вас имеется 100 долларов, и вы можете купить 100 животных. Собаки стоят по 15 долларов, коты по 1 доллару, мыши по 25 центов. Вы должны купить хотя бы по одному животному каждого вида. Найдите решение этого паззла.

In [ ]:
# наши переменные (количества животных)
dog, cat, mouse = Ints('dog cat mouse')
solve(dog >= 1,   # не менее одной собаки
      cat >= 1,   # не менее одного кота
      mouse >= 1, # не менее одной мыши

      dog + cat + mouse == 100, # всего ровно сто животных

      # У нас ровно 100 долларов (10000 центов):
      #   пёс стоит 15 долларов (1500 центов),
      #   кот стоит 1 доллар (100 центов),
      #   мышь стоит 25 центов
      1500 * dog + 100 * cat + 25 * mouse == 10000)

## 8.2. Судоку

Судоку -- это классическая и очень популярная головоломка, которая стала самым популярным паззлом при обучении солверам.

Есть квадрат 9x9 клеток, и надо заполнить его цифрами от 1 до 9 так, чтобы в каждом столбце, в каждой строке и в каждом квадрате 3x3 клетки содержались все цифры от 1 до 9.

In [ ]:

# исходная матрица 9x9 -- стандартный список списков
X = [ [ Int("x_%s_%s" % (i+1, j+1)) for j in range(9) ]
      for i in range(9) ]

# формируем ограничения:

# в каждой ячейке значение от 1 до 9
cells_c  = [ And(1 <= X[i][j], X[i][j] <= 9)
             for i in range(9) for j in range(9) ]


Как проверить, что в столбце/строке все числа различны? В этом поможет функция Z3 `Distinct()`. Вызываемая, в частности, в генераторе списка, она обеспечивает различие всех её аргументов в этом списке.

In [ ]:
# в каждой строке матрицы имеются все цифры от 1 до 9
rows_c   = [ Distinct(X[i]) for i in range(9) ]

# в каждом столбце матрицы имеются все цифры от 1 до 9
cols_c   = [ Distinct([ X[i][j] for i in range(9) ])
             for j in range(9) ]

# каждый квадрат 3x3 содержит все цифры от 1 до 9
sq_c     = [ Distinct([ X[3*i0 + i][3*j0 + j]
                        for i in range(3) for j in range(3) ])
             for i0 in range(3) for j0 in range(3) ]

# формируем общее ограничение на саму матрицу (набор списков)
sudoku_c = cells_c + rows_c + cols_c + sq_c

# в ячейках, которые надо заполнить, оставим 0
instance = ((0,0,0,0,0,4,0,3,0),
            (0,0,0,5,1,0,0,0,7),
            (0,8,9,0,0,0,0,4,0),
            (0,0,0,0,0,0,2,0,8),
            (0,6,0,2,0,1,0,5,0),
            (1,0,2,0,0,0,0,0,0),
            (0,7,0,0,0,0,5,2,0),
            (9,0,0,0,6,5,0,0,0),
            (0,4,0,9,7,0,0,0,0))

# добавим условие для результата:

# идея, что для ячеек, где уже имеется "итоговое" значение,
# мы добавляем в модель ограничение, чтобы в результате в соответствующей ячейке
# оставалось именно такое число;

# а остальные ячейки с нулями никак не обрабатываем (модель свободна в подборе значений для них)
instance_c = [ If(instance[i][j] == 0, # в ячейке 0 ?
                  True, # ограничение не добавляем
                  X[i][j] == instance[i][j]) # требуем, чтобы в итоге значение было точно таким
               for i in range(9) for j in range(9) ]

s = Solver()
s.add(sudoku_c + instance_c) # загружаем в солвер итоговый список условий

if s.check() == sat: # решено?
    m = s.model()
    r = [ [ m.evaluate(X[i][j]) for j in range(9) ] # вычисляем нужные значения
          for i in range(9) ]
    print_matrix(r)
else:
    print("не получилось...")

def n_solutions(n): # вывод первых n решений
    s = Solver()
    s.add(sudoku_c + instance_c)
    i = 0
    while s.check() == sat and i < n:
        m = s.model()
        r = [[ m.evaluate(X[i][j]) for j in range(9)] for i in range(9)]
        print_matrix(r)
        fml = And([X[i][j] == m.evaluate(X[i][j]) for i in range(9) for j in range(9)])
        s.add(Not(fml)) # накладываем дополнительное ограничение -- чтобы данное решение не повторялось на будущее
        i += 1

n_solutions(10)

## 8.3. Восемь ферзей

Ещё одна классическая задачка, которую очень любят решать на курсах по функциональному и декларативному программированию. На шахматной доске надо расставить восемь ферзей так, чтобы никто друг друга не атаковал.

В терминах условий Z3, нам по сути требуется задать всего одно ограничение: чтобы никакие два ферзя не располагались в одном столбце, строке или диагонали. Солвер на то и солвер, чтобы решать такие задачи для нас автоматически.


In [ ]:
# Сперва представим в нашей системе ферзей.
# Пусть каждый ферзь будет просто уникальное целое число от 1 до 8:
Queens = [ Int('Q_%i' % (i + 1)) for i in range(8) ]

# Теперь задаём ограничения на их расположение на доске.

# Каждый ферзь должен быть уникальным в каждой строке.
# Для этого просто потребуем, чтобы значения ферзей укладывались в диапазон 1..8,
# а так как ферзей восемь, им "автоматически" придётся быть различными:
val_c = [ And(1 <= Queens[i], Queens[i] <= 8) for i in range(8) ]

# Уникальность ферзей в столбцах зададим с помощью Distinct():
col_c = [ Distinct(Queens) ]

# Столбцы и строки тут, конечно, условные.
# Просто считаем их ортогональными, без привязки к реальным направлениям.

# Ограничения по диагоналям:
diag_c = [ If(i == j,
              True,
              And(Queens[i] - Queens[j] != i - j, Queens[i] - Queens[j] != j - i))
           for i in range(8) for j in range(i) ]

solve(val_c + col_c + diag_c) # покажите решение!

## 8.4. Словесный паззл

Один из классических типов головоломок -- это нахождение цифр, соответствующих буквам словесного выражения. Например, имеется выражение `send + more = money`, и надо вместо букв подставить подходящие цифры (одинаковые буквы -- это одинаковые цифры), чтобы оно получилось истинным.

In [ ]:
digits = Ints('s e n d m o r y') # задаём все буквы выражения как отдельные переменные
s,e,n,d,m,o,r,y = digits # определим их и непосредственно в питоне

# переводим слова в числа
# с учётом десятичной позиции в соответствующем числе
send = Sum([10**(3-i) * d for i,d in enumerate([s,e,n,d])])
more = Sum([10**(3-i) * d for i,d in enumerate([m,o,r,e])])
money = Sum([10**(4-i) * d for i,d in enumerate([m,o,n,e,y])])

# print(send)

solver = Solver()
solver.add([s > 0, m > 0]) # первые цифры каждого числа должны быть ненулевые
solver.add( [And(0 <= d, d <= 9) for d in digits]  ) # диапазоны остальных чисел -- одноразрядные цифры

# все "цифры" s,e,n,d,m,o,r,y должны отличаться друг от друга
solver.add( [ d1 != d2 for d1 in digits for d2 in digits if not eq(d1,d2)]  )

# основное ограничение задачи
solver.add(send + more == money) # :-)
solver.check()

m = solver.model()
print(" "+str(m.evaluate(send)))
print(" "+str(m.evaluate(more)))
print("-----")
print(m.evaluate(money))

# 9. Прикладное применение: управление зависимостями между пакетами

Если у вас есть опыт разбирательства со взаимосвязями между версиями пакетов в крупном проекте, вы наверняка сталкивались при этом с множеством проблем. В общем случае поиск подходящего (даже не минимального) набора зависимостей для успешной установки корневого пакета -- NP-полная задача. То есть поиск такого решения может занять непредсказуемое время (и довольно часто, очень большое):
https://vk.com/wall-152484379_278

Однако солверы для этой темы применяются весьма активно и достаточно успешно.

Пусть каждый дистрибутив содержит некоторый файл с мета-данными, который задаёт список зависимостей для каждого пакета в дистрибутиве (имя пакета, версия, ...). Формально это задаётся так, что в мета-данных имеются зависимости (какие другие пакеты должны присутствовать в системе) и конфликтные условия (какие другие пакеты не должны присутствовать).

Такую хорошо сформулированную задачу Z3 решает легко, если решение существует. Идея системы зависимостей будет такая, что мы определяем булеву переменную для каждого пакета. Она равна true, если соответствующий пакет должен быть в системе, и false, если соответствующий пакет не должен быть в системе.


In [ ]:
a, b, c, d, e, f, g, z = Bools('a b c d e f g z') # список всех пакетов (и нужных, и ненужных)

# Сформировать условие для зависимости между пакетами
def DependsOn(pack, deps):
   return And([ Implies(pack, dep) for dep in deps ]) # помните импликацию (причина-следствие)?

# Например, DependsOn(a, [b, c, z]) сгенерирует ограничение
# And(Implies(a, b), Implies(a, c), Implies(a, z))

# Если устанавливается пакет a, то необходимо также установить пакеты b, c и z.
print(DependsOn(a, [b, c, z]))

# Если пакет d конфликтует с пакетом e, нужно выбрать только один из них.
def Conflict(p1, p2):
    return Or(Not(p1), Not(p2))

# Пример. Обратите внимание, что сам список зависимых пакетов
# может формироваться не линейно, а с логическими условиями.
# Например, пакет c зависит от d или e (достаточно любого из них).

solve(DependsOn(a, [b, c, z]),
      DependsOn(b, [d]),
      DependsOn(c, [Or(d, e), Or(f, g)]),
      Conflict(d, e),
      a, z)

# Решение: пакеты g и e включать в проект не надо.


А если, например, зависимости пакета c изменить так: `DependsOn(c, [And(d, e), Or(f, g)])` -- (требуем, чтобы обязательно были пакеты d и e), то решения, естественно, не найдётся (так как эти пакеты также конфликтуют).

In [ ]:
# вывод результирующего набора пакетов в более наглядном виде
def install_check(*problem):
    s = Solver()
    s.add(*problem)
    if s.check() == sat:
        m = s.model()
        r = []
        for x in m:
            if is_true(m[x]):
                # x is a Z3 declaration
                # x() returns the Z3 expression
                # x.name() returns a string
                r.append(x())
        print(r)
    else:
        print("неверный инсталляционный профиль")

print("Вариант 1")
install_check(DependsOn(a, [b, c, z]),
              DependsOn(b, [d]),
              DependsOn(c, [Or(d, e), Or(f, g)]),
              Conflict(d, e),
              Conflict(d, g),
              a, z)

print("Вариант 2")
install_check(DependsOn(a, [b, c, z]),
              DependsOn(b, [d]),
              DependsOn(c, [Or(d, e), Or(f, g)]),
              Conflict(d, e),
              Conflict(d, g),
              a, z, g)
